In [1]:
%load_ext dotenv
%dotenv

In [2]:
import os

INPUT_CSV_TRANSFORM = os.getenv("INPUT_CSV_TRANSFORM")

assert INPUT_CSV_TRANSFORM != None, "Missing INPUT_CSV_TRANSFORM"

In [3]:
import transform_investing_spark as ti

from common.constants import BALANCE_ARRAY_COLS, INCOME_ARRAY_COLS

%load_ext autoreload
%autoreload 1
%aimport common, transform_investing_spark

Spark session:

In [4]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master('local[*]')
    .config('spark.ui.port', '5050')
    .appName('transform_investing')
    .getOrCreate()
)

22/10/15 15:14:52 WARN Utils: Your hostname, xenial64 resolves to a loopback address: 127.0.2.1; using 10.0.2.15 instead (on interface enp0s3)
22/10/15 15:14:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/15 15:14:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Reading data:

In [5]:
raw_df = spark.read.csv(INPUT_CSV_TRANSFORM, header=True)
raw_df.show()

+-------------+---------+----------------+--------------------+----------------+--------------------+
|     stock_id|source_id|       page_name|     extraction_time|          metric|               value|
+-------------+---------+----------------+--------------------+----------------+--------------------+
|     CIB:NYSE|investing|         general|2022-10-14T18:52:...|        currency|                 USD|
|     CIB:NYSE|investing|         general|2022-10-14T18:52:...|           price|               25.09|
|     CIB:NYSE|investing|         general|2022-10-14T18:52:...|          shares|         961,827,000|
|  AAPL:NASDAQ|investing|   balance-sheet|2022-10-14T18:52:...|      date_years| 2022,2022,2021,2021|
|  AAPL:NASDAQ|investing|         general|2022-10-14T18:52:...|        currency|                 USD|
|     CIB:NYSE|investing|   balance-sheet|2022-10-14T18:52:...|      date_years| 2022,2022,2021,2021|
|      EC:NYSE|investing|         general|2022-10-14T18:52:...|        currency|  

In [6]:
raw_df.filter('source_id == "google"').show()

+-----------+---------+---------+--------------------+--------+------+
|   stock_id|source_id|page_name|     extraction_time|  metric| value|
+-----------+---------+---------+--------------------+--------+------+
|   CIB:NYSE|   google|    quote|2022-10-14T18:53:...|   price|    25|
|   CIB:NYSE|   google|    quote|2022-10-14T18:53:...|pe_ratio|  4.66|
|AAPL:NASDAQ|   google|    quote|2022-10-14T18:53:...|   price|138.69|
|AAPL:NASDAQ|   google|    quote|2022-10-14T18:53:...|pe_ratio| 22.91|
|    EC:NYSE|   google|    quote|2022-10-14T18:53:...|   price|   9.2|
|    EC:NYSE|   google|    quote|2022-10-14T18:53:...|pe_ratio|  3.21|
+-----------+---------+---------+--------------------+--------+------+



In [7]:
investing_df = (
    raw_df
    .filter("source_id == 'investing'")
    .drop("source_id")
)
investing_df.show()

+-------------+----------------+--------------------+----------------+--------------------+
|     stock_id|       page_name|     extraction_time|          metric|               value|
+-------------+----------------+--------------------+----------------+--------------------+
|     CIB:NYSE|         general|2022-10-14T18:52:...|        currency|                 USD|
|     CIB:NYSE|         general|2022-10-14T18:52:...|           price|               25.09|
|     CIB:NYSE|         general|2022-10-14T18:52:...|          shares|         961,827,000|
|  AAPL:NASDAQ|   balance-sheet|2022-10-14T18:52:...|      date_years| 2022,2022,2021,2021|
|  AAPL:NASDAQ|         general|2022-10-14T18:52:...|        currency|                 USD|
|     CIB:NYSE|   balance-sheet|2022-10-14T18:52:...|      date_years| 2022,2022,2021,2021|
|      EC:NYSE|         general|2022-10-14T18:52:...|        currency|                 USD|
|  AAPL:NASDAQ|   balance-sheet|2022-10-14T18:52:...|date_days_months|25/06,26/0

In [8]:
investing_df.select("page_name").distinct().show()

+----------------+
|       page_name|
+----------------+
|   balance-sheet|
|         general|
|income-statement|
+----------------+



Page: `general`

In [9]:
general_df = ti.get_pivoted_page_df(investing_df, "general")
general_df.show()

+-------------+---------+--------------------+--------+--------+--------------+
|     stock_id|page_name|     extraction_time|currency|   price|        shares|
+-------------+---------+--------------------+--------+--------+--------------+
|  AAPL:NASDAQ|  general|2022-10-14T18:52:...|     USD|  138.52|16,070,752,000|
|BCOLOMBIA:BVC|  general|2022-10-14T18:52:...|     COP|34,200.0|   961,827,000|
|     CIB:NYSE|  general|2022-10-14T18:52:...|     USD|   25.09|   961,827,000|
|      EC:NYSE|  general|2022-10-14T18:52:...|     USD|    9.22|41,116,694,690|
|ECOPETROL:BVC|  general|2022-10-14T18:52:...|     COP| 2,163.0|41,116,694,690|
| PFBCOLOM:BVC|  general|2022-10-14T18:53:...|     COP|28,560.0|   961,827,000|
+-------------+---------+--------------------+--------+--------+--------------+



In [10]:
from pyspark.sql.functions import regexp_replace

general_df = (
    general_df
    .withColumn("price", regexp_replace("price", ",", ""))
    .withColumn("shares", regexp_replace("shares", ",", ""))
)
general_df.show()

+-------------+---------+--------------------+--------+-------+-----------+
|     stock_id|page_name|     extraction_time|currency|  price|     shares|
+-------------+---------+--------------------+--------+-------+-----------+
|  AAPL:NASDAQ|  general|2022-10-14T18:52:...|     USD| 138.52|16070752000|
|BCOLOMBIA:BVC|  general|2022-10-14T18:52:...|     COP|34200.0|  961827000|
|     CIB:NYSE|  general|2022-10-14T18:52:...|     USD|  25.09|  961827000|
|      EC:NYSE|  general|2022-10-14T18:52:...|     USD|   9.22|41116694690|
|ECOPETROL:BVC|  general|2022-10-14T18:52:...|     COP| 2163.0|41116694690|
| PFBCOLOM:BVC|  general|2022-10-14T18:53:...|     COP|28560.0|  961827000|
+-------------+---------+--------------------+--------+-------+-----------+



Page: `balance`

In [11]:
balance_df = ti.get_pivoted_page_df(investing_df, "balance-sheet")
balance_df.show()

+-------------+-------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|     stock_id|    page_name|     extraction_time|    date_days_months|         date_years|              equity|         money_units|
+-------------+-------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|  AAPL:NASDAQ|balance-sheet|2022-10-14T18:52:...|25/06,26/03,25/12...|2022,2022,2021,2021|58107,67399,71932...|* In Millions of ...|
|BCOLOMBIA:BVC|balance-sheet|2022-10-14T18:53:...|30/06,31/03,31/12...|2022,2022,2021,2021|35099020,30200262...|* In Millions of ...|
|     CIB:NYSE|balance-sheet|2022-10-14T18:52:...|30/06,31/03,31/12...|2022,2022,2021,2021|35099020,30200262...|* In Millions of ...|
|      EC:NYSE|balance-sheet|2022-10-14T18:52:...|30/06,31/03,31/12...|2022,2022,2021,2021|-,89144201,717330...|* In Millions of ...|
|ECOPETROL:BVC|balance-sheet|2022-10-14T18:52:...|30/06,31/03,

In [12]:
balance_df = ti.transform_money_units(balance_df)
balance_df.show()

+-------------+-------------+--------------------+--------------------+-------------------+--------------------+--------+--------+
|     stock_id|    page_name|     extraction_time|    date_days_months|         date_years|              equity|  factor|currency|
+-------------+-------------+--------------------+--------------------+-------------------+--------------------+--------+--------+
|  AAPL:NASDAQ|balance-sheet|2022-10-14T18:52:...|25/06,26/03,25/12...|2022,2022,2021,2021|58107,67399,71932...|Millions|     USD|
|BCOLOMBIA:BVC|balance-sheet|2022-10-14T18:53:...|30/06,31/03,31/12...|2022,2022,2021,2021|35099020,30200262...|Millions|     COP|
|     CIB:NYSE|balance-sheet|2022-10-14T18:52:...|30/06,31/03,31/12...|2022,2022,2021,2021|35099020,30200262...|Millions|     COP|
|      EC:NYSE|balance-sheet|2022-10-14T18:52:...|30/06,31/03,31/12...|2022,2022,2021,2021|-,89144201,717330...|Millions|        |
|ECOPETROL:BVC|balance-sheet|2022-10-14T18:52:...|30/06,31/03,31/12...|2022,2022,20

In [13]:
balance_df = ti.transform_from_arrays(balance_df, BALANCE_ARRAY_COLS)
balance_df.show()

+-------------+-------------+--------------------+--------+--------+----------+----------------+--------+
|     stock_id|    page_name|     extraction_time|  factor|currency|date_years|date_days_months|  equity|
+-------------+-------------+--------------------+--------+--------+----------+----------------+--------+
|  AAPL:NASDAQ|balance-sheet|2022-10-14T18:52:...|Millions|     USD|      2022|           25/06|   58107|
|  AAPL:NASDAQ|balance-sheet|2022-10-14T18:52:...|Millions|     USD|      2022|           26/03|   67399|
|  AAPL:NASDAQ|balance-sheet|2022-10-14T18:52:...|Millions|     USD|      2021|           25/12|   71932|
|  AAPL:NASDAQ|balance-sheet|2022-10-14T18:52:...|Millions|     USD|      2021|           25/09|   63090|
|BCOLOMBIA:BVC|balance-sheet|2022-10-14T18:53:...|Millions|     COP|      2022|           30/06|35099020|
|BCOLOMBIA:BVC|balance-sheet|2022-10-14T18:53:...|Millions|     COP|      2022|           31/03|30200262|
|BCOLOMBIA:BVC|balance-sheet|2022-10-14T18:53:

In [14]:
balance_df = ti.transform_date(balance_df)
balance_df.show()

+-------------+-------------+--------------------+--------+--------+--------+----------+
|     stock_id|    page_name|     extraction_time|  factor|currency|  equity|      date|
+-------------+-------------+--------------------+--------+--------+--------+----------+
|  AAPL:NASDAQ|balance-sheet|2022-10-14T18:52:...|Millions|     USD|   58107|2022/25/06|
|  AAPL:NASDAQ|balance-sheet|2022-10-14T18:52:...|Millions|     USD|   67399|2022/26/03|
|  AAPL:NASDAQ|balance-sheet|2022-10-14T18:52:...|Millions|     USD|   71932|2021/25/12|
|  AAPL:NASDAQ|balance-sheet|2022-10-14T18:52:...|Millions|     USD|   63090|2021/25/09|
|BCOLOMBIA:BVC|balance-sheet|2022-10-14T18:53:...|Millions|     COP|35099020|2022/30/06|
|BCOLOMBIA:BVC|balance-sheet|2022-10-14T18:53:...|Millions|     COP|30200262|2022/31/03|
|BCOLOMBIA:BVC|balance-sheet|2022-10-14T18:53:...|Millions|     COP|32234347|2021/31/12|
|BCOLOMBIA:BVC|balance-sheet|2022-10-14T18:53:...|Millions|     COP|30261139|2021/30/09|
|     CIB:NYSE|balanc

Page: `income`

In [15]:
import pyspark.sql.functions as F

income_df = ti.get_pivoted_page_df(investing_df, "income-statement")

# `id` for added for demo purposes
income_df = income_df.withColumn("id", F.monotonically_increasing_id())

income_df.show(truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------------
 stock_id          | AAPL:NASDAQ                                       
 page_name         | income-statement                                  
 extraction_time   | 2022-10-14T18:52:59.347448                        
 date_days_months  | 25/06,26/03,25/12,25/09                           
 date_years        | 2022,2022,2021,2021                               
 gross_profits     | 35885,42559,54243,35174                           
 money_units       | * In Millions of USD (except for per share items) 
 net_incomes       | 19442,25010,34630,20551                           
 operating_incomes | 23076,29979,41488,23786                           
 reveneus          | 82959,97278,123945,83360                          
 id                | 0                                                 
-RECORD 1--------------------------------------------------------------
 stock_id          | BCOLOMBIA:BVC                              

In [16]:
income_df = ti.transform_money_units(income_df)
income_df.show(truncate=False, vertical=True)

-RECORD 0-------------------------------------------
 stock_id          | AAPL:NASDAQ                    
 page_name         | income-statement               
 extraction_time   | 2022-10-14T18:52:59.347448     
 date_days_months  | 25/06,26/03,25/12,25/09        
 date_years        | 2022,2022,2021,2021            
 gross_profits     | 35885,42559,54243,35174        
 net_incomes       | 19442,25010,34630,20551        
 operating_incomes | 23076,29979,41488,23786        
 reveneus          | 82959,97278,123945,83360       
 id                | 0                              
 factor            | Millions                       
 currency          | USD                            
-RECORD 1-------------------------------------------
 stock_id          | BCOLOMBIA:BVC                  
 page_name         | income-statement               
 extraction_time   | 2022-10-14T18:53:00.691773     
 date_days_months  | 30/06,31/03,31/12,30/09        
 date_years        | 2022,2022,2021,2021      

In [17]:
income_df = ti.transform_from_arrays(income_df, INCOME_ARRAY_COLS)
income_df.show(5, truncate=False, vertical=True)

-RECORD 0---------------------------------------
 stock_id          | AAPL:NASDAQ                
 page_name         | income-statement           
 extraction_time   | 2022-10-14T18:52:59.347448 
 id                | 0                          
 factor            | Millions                   
 currency          | USD                        
 date_years        | 2022                       
 date_days_months  | 25/06                      
 gross_profits     | 35885                      
 net_incomes       | 19442                      
 operating_incomes | 23076                      
 reveneus          | 82959                      
-RECORD 1---------------------------------------
 stock_id          | AAPL:NASDAQ                
 page_name         | income-statement           
 extraction_time   | 2022-10-14T18:52:59.347448 
 id                | 0                          
 factor            | Millions                   
 currency          | USD                        
 date_years        |

In [18]:
income_df.count()

24

In [19]:
income_df = ti.transform_date(income_df)
income_df.show(5, truncate=False, vertical=True)

-RECORD 0---------------------------------------
 stock_id          | AAPL:NASDAQ                
 page_name         | income-statement           
 extraction_time   | 2022-10-14T18:52:59.347448 
 id                | 0                          
 factor            | Millions                   
 currency          | USD                        
 gross_profits     | 35885                      
 net_incomes       | 19442                      
 operating_incomes | 23076                      
 reveneus          | 82959                      
 date              | 2022/25/06                 
-RECORD 1---------------------------------------
 stock_id          | AAPL:NASDAQ                
 page_name         | income-statement           
 extraction_time   | 2022-10-14T18:52:59.347448 
 id                | 0                          
 factor            | Millions                   
 currency          | USD                        
 gross_profits     | 42559                      
 net_incomes       |

Stop Spark session:

In [20]:
spark.stop()